# Customer Analytics: Capstone Programming Assignment

<br>

*Austin Koenig*

---

## Table of Contents

---

<a id="toc"></a>

1.1 [Table of Contents](#toc)  
1.2 [Introduction](#intro)  
1.3 [Customer Churn](#churn)  
&nbsp;&nbsp;&nbsp;&nbsp;1.3.1 [Business Problem](#churn-prob)  
&nbsp;&nbsp;&nbsp;&nbsp;1.3.2 [Exploratory Analysis](#churn-explore)  
&nbsp;&nbsp;&nbsp;&nbsp;1.3.3 [Modeling](#churn-modeling)  
&nbsp;&nbsp;&nbsp;&nbsp;1.3.4 [Results & Insight](#churn-results)  
1.4 [Customer Value](#churn)  
&nbsp;&nbsp;&nbsp;&nbsp;1.4.1 [Business Problem](#churn-prob)  
&nbsp;&nbsp;&nbsp;&nbsp;1.4.2 [Exploratory Analysis](#churn-explore)  
&nbsp;&nbsp;&nbsp;&nbsp;1.4.3 [Modeling](#churn-modeling)  
&nbsp;&nbsp;&nbsp;&nbsp;1.4.4 [Results & Insight](#churn-results)  
1.5 [Campaign Effectiveness](#churn)  
&nbsp;&nbsp;&nbsp;&nbsp;1.5.1 [Business Problem](#churn-prob)  
&nbsp;&nbsp;&nbsp;&nbsp;1.5.2 [Exploratory Analysis](#churn-explore)  
&nbsp;&nbsp;&nbsp;&nbsp;1.5.3 [Modeling](#churn-modeling)  
&nbsp;&nbsp;&nbsp;&nbsp;1.5.4 [Results & Insight](#churn-results)  
1.6 [References](#refs)


## Introduction

<a id="intro"></a>

---

This project is effectively the combination of three smaller projects dictated by the datasets given to us in the class. For each of the datasets, there will be a corresponding business problem defined in their respective sections, which will thereby be discussed and solutions will be put forth in response. Then, we will conclude with some statements about what we've learned about customers and marketing across these analyses.

To conclude the introduction, we will import the Python libraries which are used in this project.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

np.random.seed(12)
pd.set_option('display.max_columns', 50)

## Customer Churn

<a id="churn"></a>

---

Customer churn is generally thought as the likelihood of whether or not a customer will remain a client of a subscription service. It is a simple but important idea for businesses in nearly every industry. Generally, customer churn is associated with the particular data that the company in question has. 

<br>

### Business Problem

<a id="churn-prob"></a>

In our case, we have a dataset containing several pieces of data on approximately 7000 customers (including whether or not they've churned) over the course of some time period. Most of the features collected are categorical, encompassing properties such as gender, type of internet service, etc.

In this project, we build a solution that predicts the probability of a customer churning. This probability will act as a metric to determine the degree of interaction the company should administer to the potential churners. Studies have shown that email is one of the most effective ways to engage with customers [1]. This is useful to know since emails are some of the easiest pieces of media to personalize to customers. Thus, we should relate the probability of churn to the type of email that is sent to the customer. Furthermore, there should be some customization to the emails we send to different types of customers. However, catering to the wants and needs of every single customer (of which there are over 7000) is tedious and unnecessary. Instead, we should split them into groups based on the data we have on them. For instance, we could personalize emails by contract duration, whether the customer is a senior, whether they have a partner or dependents, and which particular services they are using. For each of the categorical combinations of these groups, we can split the customers into different *churn categories* that are defined by which interval their probability of churn falls into.<br><br>

> **Churn Categories:**
> 
> - Green: $0\leq p<1/3$; Those with the lowest probability of churn
> - Yellow: $1/3\leq p<2/3$; Those with a moderate probability of churn
> - Red: $2/3\leq p<1$; Those with a high probability of churn

The green category consists of loyal customers who are not likely to unsubscribe whereas the red category contains customers who are unhappy or having trouble with the product/service and are therefore very likely to churn. The yellow category encapsulates those who may or may not churn.

There are several definitions of churn [2], but here we will define churn as a rate:

> **Definition 1:** For any time period, $T$, the churn rate is the percentage of customers who churn during $T$.

We prefer a rate since it is a lot easier to comprehend at a glance, whereas using simply the number of customers who churn also requires that we know how many customers we have at the beginning of the particular time period.

We should also note that here we are not taking into account the number of customers who we gain each month. A separate dataset on new customers may give us more insight on how to grow our customer base; however, the scope of this project only deals with maintaining the current customer base.

Now, let's explore the data to see how we can work with it.

<br>

### Exploratory Analysis

<a id="churn-explore"></a>

---

The goal of this section is to understand the data through observation during data preparation and feature selection for a predictive model. First, let's import the data take a look at the first few rows to get an idea of what we're working with.

In [2]:
churn_data_raw = pd.read_csv('./data/Customer_Churn_Dataset.csv')
churn_data_raw.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


Here, we have mostly categorical features (that are rather self-explanatory) with more than two possible values. Models generally require data to be at least binary so we will have to prepare the raw data somehow. Below, we go over the steps we take to clean the dataset, which will certainly help us understand the data in more depth.

First, there were a couple of "house-keeping" issues to take care of. We don't need the `customerID` feature since our indices are unique anyway and we are not performing a study outside the scope of these customers. Secondly, one of the numeric features was actually a string, so we simply convert it to a number. Those of the keen eye will notice below that we've exempted a few features from the one-hot encoding. These features are either numeric, already binary, or the actual target feature (`Churn`). Next, we consider briefly one-hot encoding of categorical features.

Algorithms cannot very well interpret strings such as "Electronic check" or "One year", so we need to one-hot encode the data. This means that we convert multi-valued categorical features into a larger number of binary features, which increases the dimensionality of the data substantially. Fortunately, this encoding method allows us to more effectively deal with multicollinearity issues using dimensionality reduction through correlation plots and a brief regression analysis.

In [3]:
churn_data_raw = churn_data_raw.drop('customerID', axis = 1) # drop the customerID column
churn_data_raw['TotalCharges'] = pd.to_numeric(churn_data_raw['TotalCharges'], errors = 'coerce') # str to num

churn_data = pd.DataFrame()
exempt = ['tenure', 'MonthlyCharges', 'TotalCharges', 'Churn'] # exempt from one-hot encode
    
def encode(feature, label):
    global churn_data
    new_feats = pd.get_dummies(feature, prefix = label, drop_first = True)
    churn_data = pd.concat([churn_data, new_feats], axis = 1)
    
for i in range(len(churn_data_raw.columns)):
    name = churn_data_raw.columns[i]
    if name not in exempt:
        encode(churn_data_raw[name], name)
    
churn_data = pd.concat([churn_data, churn_data_raw[['tenure', 'MonthlyCharges', 'TotalCharges']]], axis = 1)
encode(churn_data_raw['Churn'], 'Churn')
churn_data = churn_data.dropna()

In [4]:
churn_data.head()

,gender_Male,SeniorCitizen_1,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_No phone service,MultipleLines_Yes,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No internet service,OnlineSecurity_Yes,OnlineBackup_No internet service,OnlineBackup_Yes,DeviceProtection_No internet service,DeviceProtection_Yes,TechSupport_No internet service,TechSupport_Yes,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure,MonthlyCharges,TotalCharges,Churn_Yes
0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,29.85,29.85,0
1,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,34,56.95,1889.50,0
2,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,2,53.85,108.15,1
3,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,45,42.30,1840.75,0
4,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,2,70.70,151.65,1


In [5]:
churn_corr_base = churn_data.corr()
churn_corr_base.style.background_gradient(cmap = 'RdBu_r', axis = None).set_precision(2)

,gender_Male,SeniorCitizen_1,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_No phone service,MultipleLines_Yes,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No internet service,OnlineSecurity_Yes,OnlineBackup_No internet service,OnlineBackup_Yes,DeviceProtection_No internet service,DeviceProtection_Yes,TechSupport_No internet service,TechSupport_Yes,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure,MonthlyCharges,TotalCharges,Churn_Yes
gender_Male,1,-0.0018,-0.0014,0.01,-0.0075,0.0075,-0.0089,-0.011,0.0047,0.0047,-0.016,0.0047,-0.013,0.0047,-0.00081,0.0047,-0.0085,0.0047,-0.0071,0.0047,-0.01,0.0078,-0.0036,-0.012,0.0016,0.00084,0.013,0.0053,-0.014,4.8e-05,-0.0085
SeniorCitizen_1,-0.0018,1,0.017,-0.21,0.0084,-0.0084,0.14,0.25,-0.18,-0.18,-0.039,-0.18,0.067,-0.18,0.06,-0.18,-0.061,-0.18,0.11,-0.18,0.12,-0.046,-0.12,0.16,-0.024,0.17,-0.15,0.016,0.22,0.1,0.15
Partner_Yes,-0.0014,0.017,1,0.45,0.018,-0.018,0.14,0.0012,-0.00029,-0.00029,0.14,-0.00029,0.14,-0.00029,0.15,-0.00029,0.12,-0.00029,0.12,-0.00029,0.12,0.083,0.25,-0.014,0.082,-0.083,-0.097,0.38,0.098,0.32,-0.15
Dependents_Yes,0.01,-0.21,0.45,1,-0.0011,0.0011,-0.024,-0.16,0.14,0.14,0.081,0.14,0.024,0.14,0.014,0.14,0.063,0.14,-0.016,0.14,-0.038,0.069,0.2,-0.11,0.061,-0.15,0.056,0.16,-0.11,0.065,-0.16
PhoneService_Yes,-0.0075,0.0084,0.018,-0.0011,1,-1,0.28,0.29,0.17,0.17,-0.092,0.17,-0.052,0.17,-0.07,0.17,-0.095,0.17,-0.021,0.17,-0.033,-0.0031,0.0044,0.017,-0.0069,0.0027,-0.0045,0.0079,0.25,0.11,0.012
MultipleLines_No phone service,0.0075,-0.0084,-0.018,0.0011,-1,1,-0.28,-0.29,-0.17,-0.17,0.092,-0.17,0.052,-0.17,0.07,-0.17,0.095,-0.17,0.021,-0.17,0.033,0.0031,-0.0044,-0.017,0.0069,-0.0027,0.0045,-0.0079,-0.25,-0.11,-0.012
MultipleLines_Yes,-0.0089,0.14,0.14,-0.024,0.28,-0.28,1,0.37,-0.21,-0.21,0.099,-0.21,0.2,-0.21,0.2,-0.21,0.1,-0.21,0.26,-0.21,0.26,-0.0036,0.11,0.16,0.06,0.084,-0.23,0.33,0.49,0.47,0.04
InternetService_Fiber optic,-0.011,0.25,0.0012,-0.16,0.29,-0.29,0.37,1,-0.47,-0.47,-0.031,-0.47,0.17,-0.47,0.18,-0.47,-0.02,-0.47,0.33,-0.47,0.32,-0.077,-0.21,0.33,-0.051,0.34,-0.31,0.018,0.79,0.36,0.31
InternetService_No,0.0047,-0.18,-0.00029,0.14,0.17,-0.17,-0.21,-0.47,1,1,-0.33,1,-0.38,1,-0.38,1,-0.34,1,-0.41,1,-0.42,0.038,0.22,-0.32,0.0019,-0.28,0.32,-0.038,-0.76,-0.37,-0.23
OnlineSecurity_No internet service,0.0047,-0.18,-0.00029,0.14,0.17,-0.17,-0.21,-0.47,1,1,-0.33,1,-0.38,1,-0.38,1,-0.34,1,-0.41,1,-0.42,0.038,0.22,-0.32,0.0019,-0.28,0.32,-0.038,-0.76,-0.37,-0.23


This correlation matrix verifies that while there are some features which correlate with churn, there is a great deal of multicollinearity. This means that many of the features are all related with each other just as they are related to churn. One way to combat this and reduce dimensionality simultaneously is to perform a recursive regression analysis to select the most predictive features of churn. In other words, we repeatedly remove each feature and test the resulting model to optimize the variance inflation factor (VIF) [3]. We perform this process until there are no more features to remove.

In [6]:
vif = {}

def optimize_vif(data, feature_list):
    global vif
    y = data['Churn_Yes']
    
    if len(feature_list) <= 1:
        return
    else:
        temp_vif = []
        for feature in feature_list:
            temp_list = feature_list.remove(feature)
            X = data[temp_list]
            model = sm.OLS(y, X)
            fit = model.fit()
            temp_vif.append(1 / )
        

d:\python 3.7\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


Let's now take a look at the summary of our feature filter regression analysis as well as a list of the features we've found to be the most indicative of churn.

In [7]:
churn_feature_filter_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Churn_Yes   R-squared:                       0.284
Model:                            OLS   Adj. R-squared:                  0.282
Method:                 Least Squares   F-statistic:                     120.9
Date:                Mon, 25 Nov 2019   Prob (F-statistic):               0.00
Time:                        14:09:37   Log-Likelihood:                -3057.9
No. Observations:                7032   AIC:                             6164.
Df Residuals:                    7008   BIC:                             6328.
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
=========================================================================================================
                                            coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
const                                     0.2737      0.103      2.658      0.008       0.072       0.476
gender_Male                              -0.0034      0.009     -0.375      0.707      -0.021       0.014
SeniorCitizen_1                           0.0445      0.013      3.419      0.001       0.019       0.070
Partner_Yes                              -0.0009      0.011     -0.079      0.937      -0.022       0.020
Dependents_Yes                           -0.0202      0.011     -1.766      0.078      -0.043       0.002
PhoneService_Yes                          0.1341      0.095      1.408      0.159      -0.053       0.321
MultipleLines_No phone service            0.1396      0.014      9.679      0.000       0.111       0.168
MultipleLines_Yes                         0.0587      0.024      2.403      0.016       0.011       0.107
InternetService_Fiber optic               0.2104      0.110      1.920      0.055      -0.004       0.425
InternetService_No                       -0.0256      0.016     -1.621      0.105      -0.057       0.005
OnlineSecurity_No internet service       -0.0256      0.016     -1.621      0.105      -0.057       0.005
OnlineSecurity_Yes                       -0.0425      0.025     -1.710      0.087      -0.091       0.006
OnlineBackup_No internet service         -0.0256      0.016     -1.621      0.105      -0.057       0.005
OnlineBackup_Yes                         -0.0113      0.024     -0.462      0.644      -0.059       0.037
DeviceProtection_No internet service     -0.0256      0.016     -1.621      0.105      -0.057       0.005
DeviceProtection_Yes                      0.0046      0.025      0.185      0.853      -0.044       0.053
TechSupport_No internet service          -0.0256      0.016     -1.621      0.105      -0.057       0.005
TechSupport_Yes                          -0.0439      0.025     -1.754      0.079      -0.093       0.005
StreamingTV_No internet service          -0.0256      0.016     -1.621      0.105      -0.057       0.005
StreamingTV_Yes                           0.0638      0.045      1.416      0.157      -0.025       0.152
StreamingMovies_No internet service      -0.0256      0.016     -1.621      0.105      -0.057       0.005
StreamingMovies_Yes                       0.0658      0.045      1.460      0.144      -0.023       0.154
Contract_One year                        -0.1056      0.014     -7.549      0.000      -0.133      -0.078
Contract_Two year                        -0.0700      0.017     -4.110      0.000      -0.103      -0.037
PaperlessBilling_Yes                      0.0449      0.010      4.495      0.000       0.025       0.064
PaymentMethod_Credit card (automatic)    -0.0061      0.014     -0.448      0.654      -0.033       0.020
PaymentMe

In [10]:
churn_useful_features.remove('const')
for f in churn_useful_features:
    print(f)

SeniorCitizen_1
MultipleLines_No phone service
MultipleLines_Yes
Contract_One year
Contract_Two year
PaperlessBilling_Yes
PaymentMethod_Electronic check
tenure
TotalCharges


In [11]:
churn_corr = churn_data[churn_useful_features].corr()
churn_corr.style.background_gradient(cmap = 'RdBu_r', axis = None).set_precision(2)

,SeniorCitizen_1,MultipleLines_No phone service,MultipleLines_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Electronic check,tenure,TotalCharges
SeniorCitizen_1,1,-0.0084,0.14,-0.046,-0.12,0.16,0.17,0.016,0.1
MultipleLines_No phone service,-0.0084,1,-0.28,0.0031,-0.0044,-0.017,-0.0027,-0.0079,-0.11
MultipleLines_Yes,0.14,-0.28,1,-0.0036,0.11,0.16,0.084,0.33,0.47
Contract_One year,-0.046,0.0031,-0.0036,1,-0.29,-0.052,-0.11,0.2,0.17
Contract_Two year,-0.12,-0.0044,0.11,-0.29,1,-0.15,-0.28,0.56,0.36
PaperlessBilling_Yes,0.16,-0.017,0.16,-0.052,-0.15,1,0.21,0.0048,0.16
PaymentMethod_Electronic check,0.17,-0.0027,0.084,-0.11,-0.28,0.21,1,-0.21,-0.06
tenure,0.016,-0.0079,0.33,0.2,0.56,0.0048,-0.21,1,0.83
TotalCharges,0.1,-0.11,0.47,0.17,0.36,0.16,-0.06,0.83,1


## References

<a id="refs"></a>

---

[1] https://www.marketo.com/analyst-and-other-reports/the-state-of-engagement/

[2] https://blog.hubspot.com/service/what-is-customer-churn

[3] https://en.wikipedia.org/wiki/Variance_inflation_factor